In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

import ast
import json
from datetime import datetime

from pyspark.sql.functions import lit
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, from_json, array_join, transform, explode_outer, to_timestamp, from_unixtime, lit
from pyspark.sql.types import ArrayType, StructType, StringType, LongType

StatementMeta(, 019b0dda-afc2-4074-9241-4fe0e9137764, 3, Finished, Available, Finished)

In [2]:
def expandir_colunas(df, colunas_alvo=None, chaves=["codigo", "nome"], separador="|", ordem_final=None):
    """
    Expande colunas que são listas de dicionários em colunas separadas com os campos desejados (ex: codigo, nome).
    Suporta colunas como string JSON ou já como StructType/ArrayType(StructType).

    Parâmetros:
        df: DataFrame Spark original
        colunas_alvo: lista de colunas a expandir (default = ['classe', 'sistema', 'formato', 'orgaoJulgador', 'assuntos'])
        chaves: chaves dentro de cada dict a extrair (ex: 'codigo', 'nome')
        separador: separador entre elementos concatenados
        ordem_final: ordem final das colunas no DataFrame (lista)

    Retorna:
        DataFrame com colunas expandidas e reorganizadas.
    """
    if colunas_alvo is None:
        colunas_alvo = ['classe', 'sistema', 'formato', 'orgaoJulgador', 'assuntos']

    for coluna in colunas_alvo:
        tipo_coluna = df.schema[coluna].dataType

        # Se for string, assume que é uma lista de dicts em string JSON
        if isinstance(tipo_coluna, StringType):
            schema_array = ArrayType(StructType().add("codigo", StringType()).add("nome", StringType()))
            df = df.withColumn(f"__parsed_{coluna}", from_json(col(coluna), schema_array))
            for chave in chaves:
                nova_coluna = f"{coluna}_{chave}"
                df = df.withColumn(nova_coluna, array_join(transform(col(f"__parsed_{coluna}"), lambda x: x[chave]), separador))

        # Se já for ArrayType(StructType(...))
        elif isinstance(tipo_coluna, ArrayType):
            if isinstance(tipo_coluna.elementType, StructType):
                for chave in chaves:
                    nova_coluna = f"{coluna}_{chave}"
                    df = df.withColumn(nova_coluna, array_join(transform(col(coluna), lambda x: x[chave]), separador))

        # Se for StructType
        elif isinstance(tipo_coluna, StructType):
            for chave in chaves:
                nova_coluna = f"{coluna}_{chave}"
                df = df.withColumn(nova_coluna, col(f"{coluna}.{chave}"))

        else:
            raise ValueError(f"Tipo não esperado para coluna {coluna}: {tipo_coluna}")

    # Reordena as colunas, se desejado
    if ordem_final:
        colunas_disponiveis = [c for c in ordem_final if c in df.columns]
        df = df.select(*colunas_disponiveis)

    return df


StatementMeta(, 019b0dda-afc2-4074-9241-4fe0e9137764, 4, Finished, Available, Finished)

In [3]:
CAMINHO_PASTA_LOTES = "Files/DATAJUD/resultado-api/lotes"
TABELA_API = "api_datajud"
TABELA_MOVIMENTOS = "movimentos_datajud"

StatementMeta(, 019b0dda-afc2-4074-9241-4fe0e9137764, 5, Finished, Available, Finished)

In [4]:
# Leitura de todos os arquivos Parquet na pasta
df_lotes = spark.read.format("parquet").load(CAMINHO_PASTA_LOTES)

print(f"Registros carregados dos lotes: {df_lotes.count()}")

StatementMeta(, 019b0dda-afc2-4074-9241-4fe0e9137764, 6, Finished, Available, Finished)

Registros carregados dos lotes: 20743476


In [5]:
# Verifica se a tabela Delta já existe
tabelas_existentes = [t.name for t in spark.catalog.listTables()]
tabela_existe = TABELA_API in tabelas_existentes

ordem_colunas = [
    'numeroProcesso', 'classe', 'classe_codigo', 'classe_nome',  
    'sistema', 'sistema_codigo', 'sistema_nome', 'formato', 'formato_codigo',
    'formato_nome', 'tribunal', 'dataHoraUltimaAtualizacao', 'grau',
    '@timestamp', 'dataAjuizamento', 'movimentos', 'id', 'nivelSigilo',
    'orgaoJulgador', 'orgaoJulgador_codigo', 'orgaoJulgador_nome',
    'assuntos', 'assuntos_codigo', 'assuntos_nome', '_id', 'data_download'
]

df_expandido = expandir_colunas(df_lotes, ordem_final=ordem_colunas)

# Adiciona coluna de data de ingestão 
df_expandido = df_expandido.withColumn("data_ingestao", lit(datetime.now().isoformat()))

if not tabela_existe:
    print(f"Criando nova tabela Delta: {TABELA_API}")
    df_expandido.write.format("delta").mode("overwrite").saveAsTable(TABELA_API)
else:
    print(f"Tabela já existe. Iniciando merge com deduplicação por _id")

    # Lê tabela existente
    df_existente = spark.table(TABELA_API).select("_id")

    # Remove duplicados (merge incremental)
    df_novos = df_expandido.join(df_existente, on="_id", how="left_anti")

    if df_novos.count() == 0:
        print("✅ Nenhum dado novo a inserir.")
    else:
        print(f"Inserindo {df_novos.count()} novos registros...")
        df_novos.write.format("delta").mode("append").saveAsTable(TABELA_API)

print("✅ Ingestão finalizada.")

StatementMeta(, 15a409ad-16d3-4cb0-b346-0596a128e0ca, 7, Finished, Available, Finished)

Criando nova tabela Delta: api_datajud
✅ Ingestão finalizada.


### TABELA DE MOVIMENTAÇÕES

In [6]:
# from pyspark.sql import DataFrame
# from pyspark.sql.functions import col, explode_outer, to_timestamp, from_unixtime, lit
# from pyspark.sql.types import LongType

def extrair_movimentos_completos(df: DataFrame) -> DataFrame:
    """
    Extrai os movimentos e seus complementos de um DataFrame Spark que contém uma coluna 'movimentos',
    estruturada como lista de structs. Retorna um DataFrame ordenado por _id e dataHora.
    """

    df_mov = df.withColumn("movimento", explode_outer("movimentos"))

    df_mov = df_mov.withColumn("complemento", explode_outer("movimento.complementosTabelados"))

    df_resultado = df_mov.select(
        col("_id"),
        col("numeroProcesso"),
        col("movimento.codigo").alias("movimentos_codigo"),
        col("movimento.nome").alias("movimentos_nome"),
        to_timestamp(col("movimento.dataHora")).alias("movimentos_dataHora"),
        col("complemento.codigo").alias("movimentos_complementosTabelados_codigo"),
        col("complemento.descricao").alias("movimentos_complementosTabelados_descricao"),
        col("complemento.valor").alias("movimentos_complementosTabelados_valor"),
        col("complemento.nome").alias("movimentos_complementosTabelados_nome"),
        col("data_download")
        
    )

    return df_resultado.orderBy(["_id", "movimentos_dataHora"], ascending=[True, False])

StatementMeta(, 019b0dda-afc2-4074-9241-4fe0e9137764, 7, Finished, Available, Finished)

In [7]:
movimentos = spark.sql("""SELECT _id, numeroProcesso, movimentos, data_download
FROM DOL_arqs_auxiliares.api_datajud""")

StatementMeta(, 019b0dda-afc2-4074-9241-4fe0e9137764, 8, Finished, Available, Finished)

In [8]:
mov_datajud = extrair_movimentos_completos(movimentos)

StatementMeta(, 019b0dda-afc2-4074-9241-4fe0e9137764, 9, Finished, Available, Finished)

In [24]:
display(mov_datajud.tail(100))

StatementMeta(, 019b0dda-afc2-4074-9241-4fe0e9137764, 12, Finished, Available, Finished)

Py4JJavaError: An error occurred while calling o7896.tailToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 25 in stage 38.0 failed 4 times, most recent failure: Lost task 25.3 in stage 38.0 (TID 7458) (vm-4d492599 executor 43): ExecutorLostFailure (executor 43 exited caused by one of the running tasks) Reason: Container from a bad node: container_1755784154804_0001_01_000044 on host: vm-4d492599. Exit status: 137. Diagnostics: [2025-08-21 14:42:29.059]Container killed on request. Exit code is 137
[2025-08-21 14:42:29.081]Container exited with a non-zero exit code 137. 
[2025-08-21 14:42:29.084]Killed by external signal
.
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:3055)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2991)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2990)
	at scala.collection.mutable.Re

In [9]:
# Verifica se a tabela Delta já existe
tabelas_existentes = [t.name for t in spark.catalog.listTables()]
tabela_mov_existe = TABELA_MOVIMENTOS in tabelas_existentes

# AJUSTA PROBLEMA DE DATAS < 01/01/1900
mov_datajud = mov_datajud.filter(col("movimentos_dataHora") >= "1900-01-01") 

mov_datajud = mov_datajud.withColumn("data_ingestao", lit(datetime.now().isoformat()))

# Define política de escrita de datas antigas
# spark.conf.set("spark.sql.parquet.datetimeRebaseModeInWrite", "CORRECTED")

if not tabela_mov_existe:
    print(f"Criando nova tabela Delta: {TABELA_MOVIMENTOS}")
    mov_datajud.write.format("delta").mode("overwrite").saveAsTable(TABELA_MOVIMENTOS)

else:
    print(f"Tabela já existe. Iniciando merge com deduplicação por múltiplas colunas")

    # Define a chave de deduplicação
    chave_deduplicacao = [
        "_id",
        'numeroProcesso',
        "movimentos_codigo",
        "movimentos_nome",
        "movimentos_dataHora"
    ]

    # Lê dados existentes apenas com as colunas de deduplicação
    df_existente = spark.table(TABELA_MOVIMENTOS).select(*chave_deduplicacao).dropDuplicates()

    # Identifica novos registros via anti join
    df_novos = mov_datajud.join(df_existente, on=chave_deduplicacao, how="left_anti")

    # Verifica se há dados novos
    if df_novos.rdd.isEmpty():
        print("✅ Nenhum dado novo a inserir.")
    else:
        total = df_novos.count()
        print(f"Inserindo {total} novos registros...")
        df_novos.write.format("delta").mode("append").saveAsTable(TABELA_MOVIMENTOS)

print("✅ Ingestão finalizada.")

StatementMeta(, 019b0dda-afc2-4074-9241-4fe0e9137764, 11, Finished, Available, Finished)

Criando nova tabela Delta: movimentos_datajud


Py4JJavaError: An error occurred while calling o7899.saveAsTable.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 73 in stage 37.0 failed 4 times, most recent failure: Lost task 73.3 in stage 37.0 (TID 7249) (vm-c2983333 executor 19): ExecutorLostFailure (executor 19 exited caused by one of the running tasks) Reason: Container from a bad node: container_1755784154804_0001_01_000020 on host: vm-c2983333. Exit status: 137. Diagnostics: [2025-08-21 14:21:20.846]Container killed on request. Exit code is 137
[2025-08-21 14:21:20.859]Container exited with a non-zero exit code 137. 
[2025-08-21 14:21:20.861]Killed by external signal
.
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:3055)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2991)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2990)
	at scala.collection.mutable.Res

In [11]:
# display(spark.sql("""
# SELECT *
# FROM DOL_arqs_auxiliares.movimentos_datajud
# limit 100"""))

StatementMeta(, f885abcb-2cc9-4f14-89ef-7565e69ca288, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f4ee6c05-4a0b-4e52-b598-a40ed010a59d)